In [ ]:
# Importando os pacotes necessários
import requests
from zipfile import ZipFile
import pandas as pd
from io import BytesIO

In [ ]:
path_dados_gov = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/'
url_paths = [
    path_dados_gov + 'ca-2019-02.csv',
    path_dados_gov + 'ca-2020-01.csv',
    path_dados_gov + 'ca-2020-02.csv',
    path_dados_gov + 'ca-2021-01.csv',
    path_dados_gov + 'ca-2021-02.csv',
    path_dados_gov + 'precos-semestrais-ca.zip',
    path_dados_gov + 'ca-2022-02.zip',
    path_dados_gov + 'ca-2023-01.zip',
    path_dados_gov + 'ca-2023-02.zip',
]

In [ ]:
def ler_dataframe_url(url, max_tentativas=3, encoding='utf-8', **kwargs):
    """
    Função para baixar e ler um arquivo CSV de uma URL, suportando diretamente arquivos CSV
    e arquivos zip contendo arquivos CSV. Retorna um DataFrame do pandas.

    Parâmetros:
    - url (str): URL do arquivo a ser lido.
    - max_tentativas (int): Número máximo de tentativas para garantir a leitura completa.
    - encoding (str): Codificação a ser usada na leitura do arquivo CSV.
    - **kwargs: Argumentos adicionais a serem passados para pd.read_csv.

    Retorno:
    - pd.DataFrame: DataFrame contendo os dados do arquivo CSV.
    - None: Em caso de falha no download ou erro na leitura do arquivo.

    Exemplo de uso:
    >>> url = "https://exemplo.com/arquivo.zip"
    >>> df = ler_dataframe_url_combined(url, sep=';')  # Argumentos adicionais podem ser especificados aqui
    >>> if df is not None:
    ...     print(df.head())
    ... else:
    ...     print("Falha ao ler o DataFrame.")

    """
    tentativas = 0
    while tentativas < max_tentativas:
        try:
            # Baixar o arquivo da URL
            response = requests.get(url)

            # Verificar se o download foi bem-sucedido (código 200)
            if response.status_code == 200:
                # Verificar o tipo de conteúdo (zip ou csv)
                if response.headers['content-type'] == 'application/zip':
                    # Extrair o conteúdo do arquivo zip
                    with ZipFile(BytesIO(response.content)) as zip_file:
                        # Assumindo que há apenas um arquivo .csv no zip, você pode pegar o primeiro da lista
                        nome_arquivo_csv = zip_file.namelist()[0]

                        # Ler o arquivo CSV em um DataFrame do pandas
                        with zip_file.open(nome_arquivo_csv) as csv_file:
                            return pd.read_csv(csv_file, encoding=encoding, **kwargs)
                elif response.headers['content-type'] == 'text/csv':
                    # Se for um arquivo CSV direto, ler em um DataFrame do pandas
                    return pd.read_csv(BytesIO(response.content), encoding=encoding, **kwargs)
                else:
                    raise ValueError(f"Formato de arquivo não suportado para a URL: {url}")

            else:
                raise requests.exceptions.RequestException(f"Falha no download. Código de status: {response.status_code} para a URL: {url}")

        except (pd.errors.ParserError, requests.exceptions.ChunkedEncodingError, ValueError) as e:
            tentativas += 1
            print(f"Tentativa {tentativas}/{max_tentativas}: Falha na leitura do arquivo. Mensagem de erro: {e}")

    print(f"Atenção: Leitura incompleta após {max_tentativas} tentativas. Alguns dados podem estar ausentes.")
    return None

In [ ]:
def ler_dataframe_url_log(url, **kwargs):
    try:
        df = ler_dataframe_url(url, **kwargs)
        if df is not None:
            print(f"Leitura bem-sucedida para {url}")
            return df
        else:
            print(f"Falha na leitura para {url}")
            return None
    except Exception as e:
        print(f"Erro durante a leitura de {url}: {e}")
        return None

def montar_base_anp_ca(url_paths, sep=';', decimal=',', encoding='latin-1', **kwargs):
    """
    Função para montar a base histórica de combustíveis automotivos da ANP disponível no portal de dados dados.gov.

    Parâmetros:
    - url_paths (list): Lista de URLs para download dos históricos semestrais.
    - sep (str): Delimitador do arquivo CSV.
    - decimal (str): Separador decimal no arquivo CSV.
    - encoding (str): Codificação a ser usada na leitura do arquivo CSV.
    - **kwargs: Argumentos adicionais a serem passados para a função ler_dataframe_url.

    Retorno:
    - pd.DataFrame: DataFrame compilando as bases de histórico.
    - None: Em caso de falha no download ou erro na leitura dos arquivos.
    """
    try:
        dfs = []

        for url in url_paths:
            df_aux = ler_dataframe_url_log(url, sep=sep, decimal=decimal, encoding=encoding, **kwargs)
            if df_aux is not None:
                df_aux.columns = [
                    'Regiao_Sigla', 'Estado_Sigla', 'Municipio', 'Revenda',
                    'CNPJ_da_Revenda', 'Nome_da_Rua', 'Numero_Rua', 'Complemento', 'Bairro',
                    'Cep', 'Produto', 'Data_da_Coleta', 'Valor_de_Venda', 'Valor_de_Compra',
                    'Unidade_de_Medida', 'Bandeira'
                ]
                dfs.append(df_aux)
            else:
                print(f"Falha na leitura do caminho: {url}")
                return None

        df = pd.concat(dfs, ignore_index=True)
        print("Concatenação bem-sucedida.")
        return df

    except Exception as e:
        print(f"Erro durante a execução da função: {e}")
        return None


In [ ]:
df = montar_base_anp_ca(url_paths, sep=';', decimal=',', encoding='latin-1')

if df is not None:
  df.info()
  # salvando amostra
  df.to_parquet('df.parquet.gzip',
              compression='gzip')
else: print("FAIL!!!")

Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2019-02.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-01.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-02.csv
Tentativa 1/3: Falha na leitura do arquivo. Mensagem de erro: ('Connection broken: IncompleteRead(1900544 bytes read, 56010580 more expected)', IncompleteRead(1900544 bytes read, 56010580 more expected))
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2021-01.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2021-02.csv
Leitura bem-sucedida para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/precos-semestrais-ca.zip
Leitura bem-sucedida par


ex de projeto neses dados
https://github.com/opablodantas/preco_combustiveis/blob/main/preco_combustiveis.ipynb

referencia dos dados
https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis
